In [ ]:
import pandas as pd

from sklearn.preprocessing import MinMaxScaler
# Montar Google Drive
drive.mount('/gdrive')

In [ ]:
# Leer set de datos
path = '/gdrive/MyDrive/Colab Notebooks/LSTM_MFT/Resultados/'
data = pd.read_csv(path + "02_data_prepared.csv", sep="|", index_col=0, parse_dates=True)
data.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 25083 entries, 2025-06-06 00:00:00 to 2025-09-01 02:10:00
Data columns (total 10 columns):
 #   Column                         Non-Null Count  Dtype  
---  ------                         --------------  -----  
 0   error_code_OK                  25083 non-null  int64  
 1   error_code_NOK                 25083 non-null  int64  
 2   error_code_PEN                 25083 non-null  int64  
 3   type_transmission_a_demanda    25083 non-null  int64  
 4   type_transmission_recoleccion  25083 non-null  int64  
 5   num_files_total                25083 non-null  int64  
 6   files_size_MB_total            25083 non-null  float64
 7   dia_semana                     25083 non-null  int64  
 8   hora                           25083 non-null  int64  
 9   total_transmisiones            25083 non-null  int64  
dtypes: float64(1), int64(9)
memory usage: 2.1 MB


In [ ]:
data.head()

,error_code_OK,error_code_NOK,error_code_PEN,type_transmission_a_demanda,type_transmission_recoleccion,num_files_total,files_size_MB_total,dia_semana,hora,total_transmisiones
2025-06-06 00:00:00,36,2,100,22,116,141,112.41278,4,0,138
2025-06-06 00:05:00,35,0,5,21,19,50,2355.75830,4,0,40
2025-06-06 00:10:00,35,0,7,22,20,36,559.76390,4,0,42
2025-06-06 00:15:00,51,0,16,20,47,50,2270.19480,4,0,67
2025-06-06 00:20:00,52,0,18,23,47,47,1001.24720,4,0,70


In [ ]:
#num_files_total y peso, no usar solo eso..
#añadir una media y desviacion movil

window_size = 12 # 1 hora

data['count_moving_mean'] = data['num_files_total'].rolling(window=window_size).mean()
data['count_moving_std'] = data['num_files_total'].rolling(window=window_size).std()
data['size_moving_mean'] = data['files_size_MB_total'].rolling(window=window_size).mean()
data['size_moving_std'] = data['files_size_MB_total'].rolling(window=window_size).std()

unsplitted = data[['num_files_total', 'count_moving_mean', 'count_moving_std', 'files_size_MB_total', 'size_moving_mean', 'size_moving_std']]
unsplitted.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 25083 entries, 2025-06-06 00:00:00 to 2025-09-01 02:10:00
Data columns (total 6 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   num_files_total      25083 non-null  int64  
 1   count_moving_mean    25072 non-null  float64
 2   count_moving_std     25072 non-null  float64
 3   files_size_MB_total  25083 non-null  float64
 4   size_moving_mean     25072 non-null  float64
 5   size_moving_std      25072 non-null  float64
dtypes: float64(5), int64(1)
memory usage: 1.3 MB


In [ ]:
unsplitted.tail()

,num_files_total,count_moving_mean,count_moving_std,files_size_MB_total,size_moving_mean,size_moving_std
2025-09-01 01:50:00,0,12.750000,44.167296,0.0,3902.539417,13518.793096
2025-09-01 01:55:00,0,12.750000,44.167296,0.0,3902.539417,13518.793096
2025-09-01 02:00:00,0,12.750000,44.167296,0.0,3902.539417,13518.793096
2025-09-01 02:05:00,1,12.833333,44.141989,0.0,3902.539417,13518.793096
2025-09-01 02:10:00,0,12.833333,44.141989,0.0,3902.539417,13518.793096


In [ ]:
val_start = unsplitted.index.max() - pd.Timedelta(weeks=2) + pd.Timedelta(minutes=5)
test_end = val_start - pd.Timedelta(minutes=5)
test_start = val_start - pd.Timedelta(weeks=2)
train_end = test_start - pd.Timedelta(minutes=5)
train_start = unsplitted.index.min() + pd.Timedelta(minutes=55)

validation = unsplitted.loc[val_start:].copy()
validation.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 4032 entries, 2025-08-18 02:15:00 to 2025-09-01 02:10:00
Data columns (total 6 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   num_files_total      4032 non-null   int64  
 1   count_moving_mean    4032 non-null   float64
 2   count_moving_std     4032 non-null   float64
 3   files_size_MB_total  4032 non-null   float64
 4   size_moving_mean     4032 non-null   float64
 5   size_moving_std      4032 non-null   float64
dtypes: float64(5), int64(1)
memory usage: 220.5 KB


In [ ]:
testing = unsplitted.loc[test_start:test_end].copy()
testing.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 4032 entries, 2025-08-04 02:15:00 to 2025-08-18 02:10:00
Data columns (total 6 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   num_files_total      4032 non-null   int64  
 1   count_moving_mean    4032 non-null   float64
 2   count_moving_std     4032 non-null   float64
 3   files_size_MB_total  4032 non-null   float64
 4   size_moving_mean     4032 non-null   float64
 5   size_moving_std      4032 non-null   float64
dtypes: float64(5), int64(1)
memory usage: 220.5 KB


In [ ]:
training = unsplitted.loc[train_start:train_end].copy()
training.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 17008 entries, 2025-06-06 00:55:00 to 2025-08-04 02:10:00
Data columns (total 6 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   num_files_total      17008 non-null  int64  
 1   count_moving_mean    17008 non-null  float64
 2   count_moving_std     17008 non-null  float64
 3   files_size_MB_total  17008 non-null  float64
 4   size_moving_mean     17008 non-null  float64
 5   size_moving_std      17008 non-null  float64
dtypes: float64(5), int64(1)
memory usage: 930.1 KB


In [ ]:
training.to_csv('./training.csv', sep='|', index=True)
testing.to_csv('./testing.csv', sep='|', index=True)
validation.to_csv('./validation.csv', sep='|', index=True)

In [ ]:
#se escala por separado para que el modelo no vea las respuestas antes de tiempo
scaler_train = MinMaxScaler(feature_range=(0, 1))
scaler_test = MinMaxScaler(feature_range=(0, 1))
scaler_val  = MinMaxScaler(feature_range=(0, 1))

In [ ]:
training_scaled = pd.DataFrame(
    scaler_train.fit_transform(training),
    index=training.index,
    columns=training.columns
)

training_scaled.head()

,num_files_total,count_moving_mean,count_moving_std,files_size_MB_total,size_moving_mean,size_moving_std
2025-06-06 00:55:00,0.000403,0.010129,0.003515,0.000406,0.049778,0.008935
2025-06-06 01:00:00,0.001103,0.009202,0.003372,0.001361,0.050744,0.008518
2025-06-06 01:05:00,0.001983,0.010351,0.003464,0.004691,0.048113,0.008032
2025-06-06 01:10:00,0.007171,0.016509,0.006634,0.005528,0.051724,0.007607
2025-06-06 01:15:00,0.006739,0.022072,0.008125,0.006166,0.050767,0.007358


In [ ]:
testing_scaled = pd.DataFrame(
    scaler_test.fit_transform(testing),
    index=testing.index,
    columns=testing.columns
)

testing_scaled.head()

,num_files_total,count_moving_mean,count_moving_std,files_size_MB_total,size_moving_mean,size_moving_std
2025-08-04 02:15:00,0.008906,0.071707,0.048273,0.005587,0.041907,0.032677
2025-08-04 02:20:00,0.004534,0.075780,0.047814,0.001231,0.042852,0.032507
2025-08-04 02:25:00,0.005864,0.080835,0.047286,0.000140,0.042954,0.032486
2025-08-04 02:30:00,0.004694,0.085110,0.046710,0.000017,0.015074,0.005813
2025-08-04 02:35:00,0.005674,0.043973,0.009149,0.000699,0.011999,0.005395


In [ ]:
validation_scaled = pd.DataFrame(
    scaler_val.fit_transform(validation),
    index=validation.index,
    columns=validation.columns
)

validation_scaled.head()

,num_files_total,count_moving_mean,count_moving_std,files_size_MB_total,size_moving_mean,size_moving_std
2025-08-18 02:15:00,0.004295,0.072167,0.068108,0.014057,0.076961,0.033345
2025-08-18 02:20:00,0.000556,0.068655,0.068580,0.003095,0.068400,0.033145
2025-08-18 02:25:00,0.000910,0.065788,0.068945,0.000034,0.068415,0.033140
2025-08-18 02:30:00,0.000994,0.062344,0.069306,0.001630,0.068072,0.033235
2025-08-18 02:35:00,0.001971,0.058730,0.069551,0.003381,0.069392,0.032897


In [ ]:
training_scaled.to_csv('./training_scaled.csv', sep='|', index=True)
testing_scaled.to_csv('./testing_scaled.csv', sep='|', index=True)
validation_scaled.to_csv('./validation_scaled.csv', sep='|', index=True)